In [1]:
# Import Nessecary Libraries

import torch
import torchvision
from torchinfo import summary
from going_modular import data_setup, engine

c:\ProgramData\anaconda3\envs\not_base_v3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.__version__

'2.0.1'

In [ ]:
'''
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
'''

'\n# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+\ntry:\n    import torch\n    import torchvision\n    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"\n    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"\n    print(f"torch version: {torch.__version__}")\n    print(f"torchvision version: {torchvision.__version__}")\nexcept:\n    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")\n    !pip3 install -U --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cu113\n    import torch\n    import torchvision\n    print(f"torch version: {torch.__version__}")\n    print(f"torchvision version: {torchvision.__version__}")\n'

In [4]:
import torch
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# Batch_size of dataset
batch_size = 32

cuda


In [5]:
# Set seeds
def set_seeds(seed: int=42):
    """Sets random sets for torch operations.

    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    """
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)

In [6]:
from pathlib import Path
image_path = Path(r'C:\Users\SameerAhamed\Downloads\Wildfire Prediction Dataset (Satellite Images)')
image_path

WindowsPath('C:/Users/SameerAhamed/Downloads/Wildfire Prediction Dataset (Satellite Images)')

In [7]:
# Setup directories
train_dir = image_path / "train"
test_dir = image_path /"test"
valid_dir = image_path /'valid'

train_dir, test_dir, valid_dir

(WindowsPath('C:/Users/SameerAhamed/Downloads/Wildfire Prediction Dataset (Satellite Images)/train'),
 WindowsPath('C:/Users/SameerAhamed/Downloads/Wildfire Prediction Dataset (Satellite Images)/test'),
 WindowsPath('C:/Users/SameerAhamed/Downloads/Wildfire Prediction Dataset (Satellite Images)/valid'))

In [8]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # "DEFAULT" = best available

# Get transforms from weights (these are the transforms used to train a particular or obtain a particular set of weights)
automatic_transforms = weights.transforms()
print(f"Automatically created transforms: {automatic_transforms}")

# Create DataLoaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=automatic_transforms,
                                                                               batch_size=batch_size)
train_dataloader, test_dataloader, class_names

Automatically created transforms: ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


(<torch.utils.data.dataloader.DataLoader at 0x2035bee20d0>,
 ['nowildfire', 'wildfire'])

In [ ]:
# Download the pretrained weights for EfficientNet_B0
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # "DEFAULT" = best available weights

# Setup the model with the pretrained weights and send it to the target device
model = torchvision.models.efficientnet_b0(weights=weights).to(device)
# model

In [ ]:
# Adjust the classifier head
from torch import nn

set_seeds()
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=len(class_names))).to(device)

In [ ]:
from torchinfo import summary

summary(model,
        input_size=(32, 3, 224, 224),
        verbose=0,
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 2]              --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   864                  True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   64                   True
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 16, 112

# Model Training starts from here. 

## NOTE: Kept code below to cause an error to stop training loop from running

In [ ]:
# Define loss function optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
..

SyntaxError: invalid syntax (304270658.py, line 1)

In [ ]:
# Setup a SummaryWriter
from tensorboardX import SummaryWriter
writer = SummaryWriter()
writer

In [ ]:
from tqdm.auto import tqdm
from typing import Dict, List, Tuple

from going_modular.engine import train_step, test_step

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for 
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]} 
    For example if training for epochs=2: 
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]} 
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        ### New: Experiment tracking ###
        # See SummaryWriter documentation
        writer.add_scalars(main_tag="Loss",
                           tag_scalar_dict={"train_loss": train_loss,
                                            "test_loss": test_loss},
                           global_step=epoch)
        
        writer.add_scalars(main_tag="Accuracy",
                           tag_scalar_dict={"train_acc": train_acc,
                                            "test_acc": test_acc},
                           global_step=epoch)
        
        writer.add_graph(model=model,
                         input_to_model=torch.randn(32, 3, 224, 224).to(device))

    # Close the writer
    writer.close()
    ### End new ### 

    # Return the filled results at the end of the epochs
    return results

In [ ]:
set_seeds()
results = train(model=model,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=2,
                device=device)

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.0939 | train_acc: 0.9671 | test_loss: 0.0346 | test_acc: 0.9895


 50%|█████     | 1/2 [04:22<04:22, 262.68s/it]

Epoch: 2 | train_loss: 0.0481 | train_acc: 0.9829 | test_loss: 0.0313 | test_acc: 0.9886


100%|██████████| 2/2 [08:45<00:00, 262.63s/it]


In [ ]:
# Save the model

from going_modular.utils import save_model

save_model(model=model,
           target_dir=r"model",
           model_name=r"EfficientNet_b0-Wildfire_Classifier.pt")

[INFO] Saving model to: model\EfficientNet_b0-Wildfire_Classifier.pt


False

In [15]:
# Load the model and
# Adjust the classifier head
import torch
from torch import nn
import torchvision
from going_modular.utils import load_model
from pathlib import Path
from going_modular import data_setup, engine

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# Batch_size of dataset
batch_size = 32

from pathlib import Path
image_path = Path(r'C:\Users\SameerAhamed\Downloads\Wildfire Prediction Dataset (Satellite Images)')
image_path

valid_dir = image_path /'valid'

weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # "DEFAULT" = best available

# Get transforms from weights (these are the transforms used to train a particular or obtain a particular set of weights)
automatic_transforms = weights.transforms()

# Loss function for prediciton
loss_fn = nn.CrossEntropyLoss()

# The code above is all the code written before but if you need to only run the prediction on
# validation dataset then start from here


model_loaded = torchvision.models.efficientnet_b0().to(device)

model_loaded.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=2)).to(device) # Hardcoded the class names

model_loaded = load_model(model_loaded, Path(r"model\EfficientNet_b0-Wildfire_Classifier.pt"))


cuda


In [16]:
# Run the Validation Dataset

valid_dataset = torchvision.datasets.ImageFolder(root=valid_dir,
                                                 transform=automatic_transforms,
                                                 is_valid_file=data_setup.check_Image)

valid_dataloader = torch.utils.data.DataLoader(
      valid_dataset,
      batch_size=batch_size,
      shuffle=False,
      pin_memory=True,
  )

In [17]:
# Predict the validation dataset
from tqdm.auto import tqdm

model_loaded.eval()

valid_loss = 0
valid_acc = 0

for X, y in tqdm(valid_dataloader):
    # Copy to device
    X , y = X.to(device), y.to(device)

    # Forward pass
    y_logits = model_loaded(X)

    # 2. Calculate and accumulate loss
    loss = loss_fn(y_logits, y)
    valid_loss += loss.item()

    # Calculate and accumulate accuracy
    test_pred_labels = y_logits.argmax(dim=1)
    valid_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

# Adjust metrics to get average loss and accuracy per batch 
valid_loss = valid_loss / len(valid_dataloader)
valid_acc = valid_acc / len(valid_dataloader)

print(f'Validation Loss- {valid_loss}')
print(f'Validation Acc - {valid_acc}')

100%|██████████| 197/197 [00:39<00:00,  5.03it/s]

Validation Loss- 0.0377111102812419
Validation Acc - 0.9874682741116751
